# Atlas Vector Search - Semantic Kernel Integration

This notebook is a companion to the [Semantic Kernel Get Started](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/semantic-kernel/) page. Refer to the page for set-up instructions and detailed explanations.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/semantic-kernel.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade semantic-kernel openai motor

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (OpenAIChatCompletion, OpenAITextEmbedding)
from semantic_kernel.connectors.memory.mongodb_atlas import MongoDBAtlasMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

In [ ]:
OPENAI_API_KEY  = "<api-key>"
ATLAS_CONNECTION_STRING = "<connection-string>"

In [ ]:
kernel = sk.Kernel()

In [ ]:
chat_service = OpenAIChatCompletion(
   service_id="chat",
   ai_model_id="gpt-3.5-turbo",
   api_key=OPENAI_API_KEY
)
embedding_service = OpenAITextEmbedding(
   ai_model_id="text-embedding-ada-002",
   api_key=OPENAI_API_KEY
)
kernel.add_service(chat_service)
kernel.add_service(embedding_service)

In [ ]:
mongodb_atlas_memory_store = MongoDBAtlasMemoryStore(
   connection_string=ATLAS_CONNECTION_STRING,
   database_name="semantic_kernel_db",
   index_name="vector_index"
)

memory = SemanticTextMemory(
   storage=mongodb_atlas_memory_store,
   embeddings_generator=embedding_service
)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

In [ ]:
async def populate_memory(kernel: sk.Kernel) -> None:
    await memory.save_information(
       collection="test", id="1", text="I am a developer"
    )
    await memory.save_information(
       collection="test", id="2", text="I started using MongoDB two years ago"
    )
    await memory.save_information(
       collection="test", id="3", text="I'm using MongoDB Vector Search with Semantic Kernel to implement RAG"
    )
    await memory.save_information(
       collection="test", id="4", text="I like coffee"
    )

print("Populating memory...")
await populate_memory(kernel)
print(kernel)

In [ ]:
# Connect to your Atlas cluster and specify the collection
client = MongoClient(ATLAS_CONNECTION_STRING)
collection = client["semantic_kernel_db"]["test"]

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
   definition={
      "fields": [
         {
         "type": "vector",
         "path": "embedding",
         "numDimensions": 1536,
         "similarity": "cosine"
         }
      ]
   },
   name="vector_index",
   type="vectorSearch"
)

collection.create_search_index(model=search_index_model)

## Semantic Search Query

In [ ]:
result = await memory.search("test", "What is my job title?")
print(f"Retrieved document: {result[0].text}, {result[0].relevance}")

## Basic RAG

In [ ]:
service_id = "chat"
settings = kernel.get_service(service_id).instantiate_prompt_execution_settings(
   service_id=service_id
)

prompt_template = """
   Answer the following question based on the given context.

   Question: {{$input}}
   Context: {{$context}}
"""

chat_prompt_template_config = PromptTemplateConfig(
   execution_settings=settings,
   input_variables=[
       InputVariable(name="input"),
       InputVariable(name="context")
   ],
   template=prompt_template
)

prompt = kernel.add_function(
   function_name="RAG",
   plugin_name="TextMemoryPlugin",
   prompt_template_config=chat_prompt_template_config,
)

question = "When did I start using MongoDB?"
results = await memory.search("test", question)
retrieved_document = results[0].text
answer = await prompt.invoke(
   kernel=kernel, input=question, context=retrieved_document
)
print(answer)